<a href="https://colab.research.google.com/github/Motunrayo244/Datacentric-AI/blob/main/Voxel51/Voxel51_Objectdetection__detection_mistakes_with_detectron_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Finding Detection Mistakes with FiftyOne using Detectron2 pretrained model for prediction

This notebook is an adoption of the tutorial notebook provided by voxel 51 on thei websites for finding detection mistakes.

A few changes were made to the default notebook, this is in order to create a standardized the variables for comparism.

In this notebook
Dataset: The dataset used is downloaded from the 118 sample images provided by cleanlab in their documentation.
Model: This notebook used the COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml default model

# Finding Detection Mistakes with FiftyOne

Annotations mistakes create an artificial ceiling on the performance of your models. However, finding these mistakes by hand is at least as arduous as the original annotation work! Enter FiftyOne.

In this tutorial, we explore how FiftyOne can be used to help you find mistakes in your object detection annotations. To detect mistakes in classification datasets, check out [this tutorial](https://voxel51.com/docs/fiftyone/tutorials/classification_mistakes.html).

We'll cover the following concepts:

- Loading your existing dataset [into FiftyOne](https://voxel51.com/docs/fiftyone/user_guide/dataset_creation/index.html)
- [Adding model predictions](https://voxel51.com/docs/fiftyone/recipes/adding_detections.html) to your dataset
- Computing insights into your dataset relating to [possible label mistakes](https://voxel51.com/docs/fiftyone/user_guide/brain.html#label-mistakes)
- Visualizing mistakes in the [FiftyOne App](https://voxel51.com/docs/fiftyone/user_guide/app.html)

**So, what's the takeaway?**

FiftyOne can help you find and correct label mistakes in your datasets, enabling you to curate higher quality datasets and, ultimately, train better models!

## Setup

If you haven't already, install FiftyOne:

In [1]:
%%capture
! pip install fiftyone

In [2]:
! python -m pip install pyyaml==5.1


! python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.2/274.2 kB 2.2 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-gehpsftc
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-gehpsftc
  Resolved https://github.com/facebookresearch/detectron2.git to commit eb96ee1d4752ff5896f623f738641fba9c755237
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50

In [3]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
torch:  2.2 ; cuda:  cu121
detectron2: 0.6


In [4]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances


In order to compute mistakenness, your dataset needs to have two [detections fields](https://voxel51.com/docs/fiftyone/user_guide/using_datasets.html#object-detection), one with your ground truth annotations and one with your model predictions.

In this notebook, we'll load the [clean lab 118 sample  dataset] (https://cleanlab-public.s3.amazonaws.com/ObjectDetectionBenchmarking/tutorial_obj/example_images.zip) The images do not have the ground truth value, so we will extract the ground truth label from the coco_dataset.

In [5]:
# Download sample images from clean lab
!wget -nc 'https://cleanlab-public.s3.amazonaws.com/ObjectDetectionBenchmarking/tutorial_obj/example_images.zip' && unzip -q -o example_images.zip

--2024-03-24 17:05:32--  https://cleanlab-public.s3.amazonaws.com/ObjectDetectionBenchmarking/tutorial_obj/example_images.zip
Resolving cleanlab-public.s3.amazonaws.com (cleanlab-public.s3.amazonaws.com)... 3.5.28.75, 54.231.232.185, 3.5.25.85, ...
Connecting to cleanlab-public.s3.amazonaws.com (cleanlab-public.s3.amazonaws.com)|3.5.28.75|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17340957 (17M) [application/zip]
Saving to: ‘example_images.zip’

example_images.zip  100%[===================>]  16.54M  50.9MB/s    in 0.3s    

2024-03-24 17:05:33 (50.9 MB/s) - ‘example_images.zip’ saved [17340957/17340957]



In [6]:
#download annotations from Coco dataset
!wget -nc http://images.cocodataset.org/annotations/annotations_trainval2017.zip && unzip -q -o annotations_trainval2017.zip

--2024-03-24 17:06:47--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.147.140, 54.231.164.201, 52.217.116.113, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.147.140|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252907541 (241M) [application/zip]
Saving to: ‘annotations_trainval2017.zip’

annotations_trainva 100%[===================>] 241.19M  90.5MB/s    in 2.7s    

2024-03-24 17:06:50 (90.5 MB/s) - ‘annotations_trainval2017.zip’ saved [252907541/252907541]



In [7]:
import random
import pickle
import fiftyone as fo
import fiftyone.zoo as foz
import json
import requests
import os

Migrating database to v0.23.7


INFO:fiftyone.migrations.runner:Migrating database to v0.23.7


In [8]:
# filter out the annotations for sample images and save their anotation information in a pickel file
directory = './example_images/'
files = os.listdir(directory)
# Filtering only the files.
sample_files = [f for f in files if os.path.isfile(directory+'/'+f)]
pickle.dump(sample_files, open('sample_images.pkl','wb'))

In [9]:
# List of selected image file names
sample_image_list = pickle.load(open("./sample_images.pkl", "rb"))

# Path to the COCO annotations file
annotations_path = './annotations/instances_val2017.json'

# Load COCO annotations
with open(annotations_path) as f:
    coco_data = json.load(f)

# Filter images and annotations
info = coco_data['info']
licenses = coco_data['licenses']
filtered_images = [img for img in coco_data['images'] if img['file_name'] in sample_image_list]
image_ids = {img['id']: img for img in filtered_images}
filtered_annotations = [ann for ann in coco_data['annotations'] if ann['image_id'] in image_ids and ann['category_id'] in [1,3,10,47,62]]

# Save filtered annotations to a new JSON file
filtered_data = {
    "info": info,
    "licenses":licenses,
    "images": filtered_images,
    "annotations": filtered_annotations,
    "categories": coco_data['categories']  # Preserve category info
}
with open('annotations/filtered_annotations.json', 'w') as f:
    json.dump(filtered_data, f, indent=4)


In [10]:
# Path to the COCO annotations file
annotations_path = './annotations/filtered_annotations.json'

# Load COCO annotations
with open(annotations_path) as f:
    coco_data = json.load(f)

# Count the number of annotations
num_annotations = len(coco_data['images'])

print(f"Number of annotations: {num_annotations}")


Number of annotations: 118


In [11]:
# make dataset into fifty one format
IMAGES_DIR = './example_images/'
# Load COCO formatted dataset
coco_dataset = fo.Dataset.from_dir(
    dataset_type=fo.types.COCODetectionDataset,
    data_path=IMAGES_DIR,
    labels_path="/content/annotations/filtered_annotations.json",
    include_id=True,
    name = 'cleanlab_coco_sample_dataset'
)

 100% |█████████████████| 118/118 [5.2s elapsed, 0s remaining, 21.4 samples/s]      


INFO:eta.core.utils: 100% |█████████████████| 118/118 [5.2s elapsed, 0s remaining, 21.4 samples/s]      


In [ ]:
print(coco_dataset)

Name:        cleanlab_coco_sample_dataset
Media type:  image
Num samples: 118
Persistent:  False
Tags:        []
Sample fields:
    id:            fiftyone.core.fields.ObjectIdField
    filepath:      fiftyone.core.fields.StringField
    tags:          fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:      fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    detections:    fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    segmentations: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    coco_id:       fiftyone.core.fields.IntField


In [12]:
# Print a sample ground truth detection
sample = coco_dataset.first()
print(sample.detections)

<Detections: {
    'detections': [
        <Detection: {
            'id': '66005e6a2696b6cb5479ca38',
            'attributes': {},
            'tags': [],
            'label': 'person',
            'bounding_box': [
                0.701359375,
                0.3757460317460317,
                0.07953125,
                0.3899365079365079,
            ],
            'mask': None,
            'confidence': None,
            'index': None,
            'supercategory': 'person',
            'iscrowd': 0,
        }>,
    ],
}>


Let's start by visualizing the dataset in the [FiftyOne App](https://voxel51.com/docs/fiftyone/user_guide/app.html):

In [13]:
# Open the dataset in the App
session = fo.launch_app(coco_dataset)


Welcome to

███████╗██╗███████╗████████╗██╗   ██╗ ██████╗ ███╗   ██╗███████╗
██╔════╝██║██╔════╝╚══██╔══╝╚██╗ ██╔╝██╔═══██╗████╗  ██║██╔════╝
█████╗  ██║█████╗     ██║    ╚████╔╝ ██║   ██║██╔██╗ ██║█████╗
██╔══╝  ██║██╔══╝     ██║     ╚██╔╝  ██║   ██║██║╚██╗██║██╔══╝
██║     ██║██║        ██║      ██║   ╚██████╔╝██║ ╚████║███████╗
╚═╝     ╚═╝╚═╝        ╚═╝      ╚═╝    ╚═════╝ ╚═╝  ╚═══╝╚══════╝ v0.23.7

If you're finding FiftyOne helpful, here's how you can get involved:

|
|  ⭐⭐⭐ Give the project a star on GitHub ⭐⭐⭐
|  https://github.com/voxel51/fiftyone
|
|  🚀🚀🚀 Join the FiftyOne Slack community 🚀🚀🚀
|  https://slack.voxel51.com
|



INFO:fiftyone.core.session.session:
Welcome to

███████╗██╗███████╗████████╗██╗   ██╗ ██████╗ ███╗   ██╗███████╗
██╔════╝██║██╔════╝╚══██╔══╝╚██╗ ██╔╝██╔═══██╗████╗  ██║██╔════╝
█████╗  ██║█████╗     ██║    ╚████╔╝ ██║   ██║██╔██╗ ██║█████╗
██╔══╝  ██║██╔══╝     ██║     ╚██╔╝  ██║   ██║██║╚██╗██║██╔══╝
██║     ██║██║        ██║      ██║   ╚██████╔╝██║ ╚████║███████╗
╚═╝     ╚═╝╚═╝        ╚═╝      ╚═╝    ╚═════╝ ╚═╝  ╚═══╝╚══════╝ v0.23.7

If you're finding FiftyOne helpful, here's how you can get involved:

|
|  ⭐⭐⭐ Give the project a star on GitHub ⭐⭐⭐
|  https://github.com/voxel51/fiftyone
|
|  🚀🚀🚀 Join the FiftyOne Slack community 🚀🚀🚀
|  https://slack.voxel51.com
|



When working with FiftyOne datasets that contain a field with `Detections`, you can create a [patches view](https://voxel51.com/docs/fiftyone/user_guide/app.html#viewing-object-patches) both through Python and directly in the FiftyOne App to view each detection as a separate sample.

### Add Prediction to the dataset
Model: preconfigured  

In [14]:
from detectron2.data import MetadataCatalog

# Get the metadata for the COCO dataset
metadata = MetadataCatalog.get("coco_2017_train")  # or "coco_2017_val", depending on your use case

# Access the list of category names
class_names = metadata.thing_classes

class_to_index = {class_name: index for index, class_name in enumerate(class_names)}
print(class_to_index)


{'person': 0, 'bicycle': 1, 'car': 2, 'motorcycle': 3, 'airplane': 4, 'bus': 5, 'train': 6, 'truck': 7, 'boat': 8, 'traffic light': 9, 'fire hydrant': 10, 'stop sign': 11, 'parking meter': 12, 'bench': 13, 'bird': 14, 'cat': 15, 'dog': 16, 'horse': 17, 'sheep': 18, 'cow': 19, 'elephant': 20, 'bear': 21, 'zebra': 22, 'giraffe': 23, 'backpack': 24, 'umbrella': 25, 'handbag': 26, 'tie': 27, 'suitcase': 28, 'frisbee': 29, 'skis': 30, 'snowboard': 31, 'sports ball': 32, 'kite': 33, 'baseball bat': 34, 'baseball glove': 35, 'skateboard': 36, 'surfboard': 37, 'tennis racket': 38, 'bottle': 39, 'wine glass': 40, 'cup': 41, 'fork': 42, 'knife': 43, 'spoon': 44, 'bowl': 45, 'banana': 46, 'apple': 47, 'sandwich': 48, 'orange': 49, 'broccoli': 50, 'carrot': 51, 'hot dog': 52, 'pizza': 53, 'donut': 54, 'cake': 55, 'chair': 56, 'couch': 57, 'potted plant': 58, 'bed': 59, 'dining table': 60, 'toilet': 61, 'tv': 62, 'laptop': 63, 'mouse': 64, 'remote': 65, 'keyboard': 66, 'cell phone': 67, 'microwave'

In [16]:
# use this to find the index of five object of interest, person, car, chair, traffic light, cup

class_to_index['car']

2

In [17]:
patches_view = coco_dataset.to_patches("detections")
print(patches_view)

Dataset:     cleanlab_coco_sample_dataset
Media type:  image
Num patches: 326
Patch fields:
    id:         fiftyone.core.fields.ObjectIdField
    sample_id:  fiftyone.core.fields.ObjectIdField
    filepath:   fiftyone.core.fields.StringField
    tags:       fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:   fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    detections: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detection)
View stages:
    1. ToPatches(field='detections', config=None)


Let's open the App and click the [patches button](https://voxel51.com/docs/fiftyone/user_guide/app.html#viewing-object-patches), then select `ground_truth` to create the same view that we created above.

In [18]:
session = fo.launch_app(coco_dataset)

In [19]:

cfg = get_cfg()
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model


# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")
cfg.MODEL.DEVICE = 'cuda'
model = DefaultPredictor(cfg)

# # model.to(device)
# model.eval()
print("Model ready")

[03/24 17:16:01 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from https://dl.fbaipublicfiles.com/detectron2/COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x/139173657/model_final_68b088.pkl ...


model_final_68b088.pkl: 421MB [00:05, 79.4MB/s]                           


Model ready


In [20]:
def detectron_to_fo(outputs, img_w, img_h):
    # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    detections = []
    instances = outputs["instances"].to("cpu")
    for pred_box, score, c in zip(
        instances.pred_boxes, instances.scores, instances.pred_classes,
    ):
      c=c.item()
      if c in [0,2,9,41,56]:
        x1, y1, x2, y2 = pred_box
        # fo_mask = mask.numpy()[int(y1):int(y2), int(x1):int(x2)]
        bbox = [float(x1)/img_w, float(y1)/img_h, float(x2-x1)/img_w, float(y2-y1)/img_h]
        detection = fo.Detection(label=classes[c], confidence=float(score), bounding_box=bbox)
        detections.append(detection)

    return fo.Detections(detections=detections)


In [21]:
from PIL import Image
from torchvision.transforms import functional as func


# Get class list
# classes = coco_dataset.default_classes
classes = {0:'person', 2:'car',41:'cup',56:'chair',9:'traffic light'}
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Add predictions to samples
with fo.ProgressBar() as pb:
    for sample in pb(coco_dataset):
        # Load image
        image = cv2.imread(sample.filepath)
        # print(image)
        # image = func.to_tensor(image).to(device)
        h, w, c = image.shape

        outputs = model(image)
        detections = detectron_to_fo(outputs, w, h)

        # Save predictions to dataset
        sample["predictions"] = detections
        sample.save()

   0% ||----------------|   0/118 [8.9ms elapsed, ? remaining, ? samples/s] 

/usr/local/lib/python3.10/dist-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


 100% |█████████████████| 118/118 [42.0s elapsed, 0s remaining, 3.4 samples/s]      


INFO:eta.core.utils: 100% |█████████████████| 118/118 [42.0s elapsed, 0s remaining, 3.4 samples/s]      


In [22]:
print(coco_dataset)

Name:        cleanlab_coco_sample_dataset
Media type:  image
Num samples: 118
Persistent:  False
Tags:        []
Sample fields:
    id:            fiftyone.core.fields.ObjectIdField
    filepath:      fiftyone.core.fields.StringField
    tags:          fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:      fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    detections:    fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    segmentations: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    coco_id:       fiftyone.core.fields.IntField
    predictions:   fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)


In [35]:
predicted_view = coco_dataset.to_patches("predictions")
print(predicted_view)

Dataset:     cleanlab_coco_sample_dataset
Media type:  image
Num patches: 383
Patch fields:
    id:          fiftyone.core.fields.ObjectIdField
    sample_id:   fiftyone.core.fields.ObjectIdField
    filepath:    fiftyone.core.fields.StringField
    tags:        fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:    fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    predictions: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detection)
View stages:
    1. ToPatches(field='predictions', config=None)


In [37]:
session = fo.launch_app(coco_dataset)

## Compute mistakenness

Now we're ready to assess the mistakenness of the ground truth detections.

We can do so by running the [compute_mistakenness()](https://voxel51.com/docs/fiftyone/api/fiftyone.brain.html#fiftyone.brain.compute_mistakenness) method from the FiftyOne Brain:

In [38]:
import fiftyone.brain as fob

# Compute mistakenness of annotations in `ground_truth` field using
# predictions from `predictions` field as point of reference
fob.compute_mistakenness(coco_dataset, "predictions", label_field="detections")

Evaluating detections...


INFO:fiftyone.utils.eval.detection:Evaluating detections...


 100% |█████████████████| 118/118 [3.1s elapsed, 0s remaining, 31.9 samples/s]          


INFO:eta.core.utils: 100% |█████████████████| 118/118 [3.1s elapsed, 0s remaining, 31.9 samples/s]          


Computing mistakenness...


INFO:fiftyone.brain.internal.core.mistakenness:Computing mistakenness...


 100% |█████████████████| 118/118 [1.7s elapsed, 0s remaining, 64.9 samples/s]          


INFO:eta.core.utils: 100% |█████████████████| 118/118 [1.7s elapsed, 0s remaining, 64.9 samples/s]          


Mistakenness computation complete


INFO:fiftyone.brain.internal.core.mistakenness:Mistakenness computation complete


The above method populates a number of fields on the samples of our dataset as well as the ground truth and predicted objects:

New ground truth object attributes (in `detection` field):

- `mistakenness` (float): A measure of the likelihood that a ground truth object's label is incorrect
- `mistakenness_loc`: A measure of the likelihood that a ground truth object's localization (bounding box) is inaccurate
- `possible_spurious`: Ground truth objects that were not matched with a predicted object and are deemed to be likely spurious annotations will have this attribute set to True

New predicted object attributes (in `predictions` field):

- `possible_missing`: If a highly confident prediction with no matching ground truth object is encountered, this attribute is set to True to indicate that it is a likely missing ground truth annotation

Sample-level fields:

- `mistakenness`: The maximum mistakenness of the ground truth objects in each sample
- `possible_spurious`: The number of possible spurious ground truth objects in each sample
- `possible_missing`: The number of possible missing ground truth objects in each sample

## Analyzing the results

Let's use FiftyOne to investigate the results.

First, let's show the samples with the most likely annotation mistakes:

In [39]:
from fiftyone import ViewField as F

# Sort by likelihood of mistake (most likely first)
mistake_view = coco_dataset.sort_by("mistakenness", reverse=True)

# Print some information about the view
print(mistake_view)

Dataset:     cleanlab_coco_sample_dataset
Media type:  image
Num samples: 118
Sample fields:
    id:                fiftyone.core.fields.ObjectIdField
    filepath:          fiftyone.core.fields.StringField
    tags:              fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:          fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    detections:        fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    segmentations:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    coco_id:           fiftyone.core.fields.IntField
    predictions:       fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    mistakenness:      fiftyone.core.fields.FloatField
    possible_missing:  fiftyone.core.fields.IntField
    possible_spurious: fiftyone.core.fields.IntField
View stages:
    1. SortBy(field_or_expr='mistakenness', reverse=True, create_index=True)


In [40]:
# Inspect some samples and detections
# This is the first detection of the first sample
print(mistake_view.first().detections.detections[0])

<Detection: {
    'id': '66005e6d2696b6cb5479cc36',
    'attributes': {},
    'tags': [],
    'label': 'chair',
    'bounding_box': [
        0.56346875,
        0.4404583333333333,
        0.055843750000000004,
        0.10791666666666666,
    ],
    'mask': None,
    'confidence': None,
    'index': None,
    'supercategory': 'furniture',
    'iscrowd': 0,
    'mistakenness': 0.04213273525238037,
    'mistakenness_loc': 0.20996548376646346,
}>


Let's use the App to visually inspect the results:

In [41]:
# Open new App window
session.show()

In [42]:
# Show the samples we processed in rank order by the mistakenness
session.view = mistake_view

Another useful query is to find all objects that have a high mistakenness, lets say > 0.50:

In [43]:
from fiftyone import ViewField as F

session.view = coco_dataset.filter_labels("detections", F("mistakenness") > 0.01)

Looking through the results, we see some annotations that may be incorrect. For example, in the image below the `goat` is labeled as a `sheep`.

We can use a similar workflow to look at objects that may be localized poorly:

In [44]:
session.view = coco_dataset.filter_labels("detections", F("mistakenness_loc") > 0.50)

One of the examples that popped up from this query is shown below. The bounding box around the person on the left side of the image is shifted too far to the right.

The `possible_missing` field can also be useful to sort by to find instances of incorrect annotations.

Similarly, `possible_spurious` can be used to find objects that the model detected that may have been missed by annotators.

In [48]:
session.view = coco_dataset.match(F("possible_missing") > 0)

An example that showed up from this search is shown above. There is an `apple` that was not annotated that the model detected.

## Tagging and resolution

Any label or collection of labels can be tagged at any time in the sample grid or expanded sample view. In the expanded sample view, individual samples can be selected by clicking on them in the media player. We can, for example, tag this `apple` prediction as `missing` and any other predictions without an associated ground truth detection.

Labels with specific tags can then be selected with [select_labels()](https://voxel51.com/docs/fiftyone/api/fiftyone.core.collections.html?highlight=select_labels#fiftyone.core.collections.SampleCollection.select_labels) stage and sent off to assist in improving the annotations with your annotation provided of choice. FiftyOne currently offers integrations for both [Labelbox](https://voxel51.com/docs/fiftyone/api/fiftyone.utils.labelbox.html) and [Scale](https://voxel51.com/docs/fiftyone/api/fiftyone.utils.scale.html).

In [ ]:
# A dataset can be filtered to only contain labels with certain tags
# Helpful for isolating labels with issues and sending off to an annotation provider
missing_ground_truth = coco_dataset.select_labels(tags="missing")

**REMEMBER**: Since you are using model predictions to guide the mistakenness process, the better your model, the more accurate the mistakenness suggestions. Additionally, using logits of confidence scores will also provide better results.

We used Faster-RCNN in this example which is quite a few years old. Using EfficientDet D7 provided much better results. For example, it was easily able to find this `snowboard` labeled as `skis`:

![skis](https://github.com/voxel51/fiftyone/blob/v0.23.5/docs/source/tutorials/images/det_mistakenness_6.png?raw=1)

In [ ]:
session.freeze() # screenshot the active App for sharing